In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import SMOTE 
from collections import Counter

from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from genetic_selection import GeneticSelectionCV

from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import NearMiss

from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Continuous, Categorical, Integer
from sklearn.model_selection import StratifiedKFold


import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('train.csv')
train.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979,0


In [3]:
test = pd.read_csv('test.csv')
test.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,2,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40532.10
1,5,2,35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45486.72
2,6,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46993.95
3,7,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,187898.61
4,9,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73649.73


In [4]:
X = train.drop('TARGET', axis=1)
y = train['TARGET']

In [5]:
# Function to resample the data
def res(X,y,ratio):
  resample = NearMiss(version=1, n_neighbors=3, sampling_strategy=ratio)
  X_sam, y_sam = resample.fit_resample(X,y)
  counter = Counter(y_sam)
  print(counter)
  return X_sam, y_sam

In [6]:
# Creating datasets with different ratio of data
X_train_1, y_train_1 = res(X, y, 1)
X_train_2, y_train_2 = res(X, y, 0.5)
X_train_3, y_train_3 = res(X, y, 0.3)

Counter({0: 3008, 1: 3008})
Counter({0: 6016, 1: 3008})
Counter({0: 10026, 1: 3008})


In [7]:
# Feature selection using Genetic Algorithm for differently sized datasets
dt1 = DecisionTreeClassifier(random_state=0)
model1 = GeneticSelectionCV(
    dt1, cv=3, verbose=0,
    scoring="accuracy", max_features=25,
    n_population=100, crossover_proba=0.2,
    mutation_proba=0.8, n_generations=100,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model1 = model1.fit(X_train_1, y_train_1)

dt2 = DecisionTreeClassifier(random_state=0)
model2 = GeneticSelectionCV(
    dt2, cv=3, verbose=0,
    scoring="accuracy", max_features=25,
    n_population=100, crossover_proba=0.2,
    mutation_proba=0.8, n_generations=100,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model2 = model2.fit(X_train_2, y_train_2)

dt3 = DecisionTreeClassifier(random_state=0)
model3 = GeneticSelectionCV(
    dt3, cv=3, verbose=0,
    scoring="accuracy", max_features=25,
    n_population=100, crossover_proba=0.2,
    mutation_proba=0.8, n_generations=100,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model3 = model3.fit(X_train_3, y_train_3)

In [8]:
# Selected features for the different datasets
col1 = model1.get_support(indices=True)
print(col1)
col2 = model2.get_support(indices=True)
print(col2)
col3 = model3.get_support(indices=True)
print(col3)

[ 17  28  38  55  76  92 131 135 178 209 222 233 247 260 265 270 271 289
 323 347 369]
[ 17  39  67  99 100 102 119 175 185 193 209 210 211 219 223 240 247 285
 303 322 333 369]
[ 47  50  62  86  94 116 147 155 170 182 194 212 229 240 249 260 294 307
 344 354 369]


In [9]:
# Feature selection using Genetic Algorithm for differently sized datasets
gb = GradientBoostingClassifier(random_state=0)
model_1 = GeneticSelectionCV(
    gb, cv=3, verbose=0,
    scoring="accuracy", max_features=25,
    n_population=100, crossover_proba=0.2,
    mutation_proba=0.8, n_generations=100,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model_1 = model_1.fit(X_train_1, y_train_1)

model_2 = GeneticSelectionCV(
    gb, cv=3, verbose=0,
    scoring="accuracy", max_features=50,
    n_population=100, crossover_proba=0.2,
    mutation_proba=0.8, n_generations=100,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model_2 = model_2.fit(X_train_2, y_train_2)

model_3 = GeneticSelectionCV(
    gb, cv=3, verbose=0,
    scoring="accuracy", max_features=50,
    n_population=100, crossover_proba=0.2,
    mutation_proba=0.8, n_generations=100,
    crossover_independent_proba=0.5,
    mutation_independent_proba=0.04,
    tournament_size=3, n_gen_no_change=10,
    caching=True, n_jobs=-1)
model_3 = model_3.fit(X_train_3, y_train_3)

In [10]:
# Selected features for the different datasets
col_1 = model_1.get_support(indices=True)
print(col_1)
col_2 = model_2.get_support(indices=True)
print(col_2)
col_3 = model_3.get_support(indices=True)
print(col3)

[  3  42  44  47  60  63  66 115 123 128 148 157 160 161 165 168 170 213
 229 260 278 286 293 335 369]
[ 10  13  30  37  44  54  55  60  65 103 121 128 135 137 138 139 147 149
 152 155 157 177 183 184 187 191 196 211 215 219 232 236 256 267 272 274
 275 280 282 290 296 302 303 310 312 315 336 340 343 369]
[ 47  50  62  86  94 116 147 155 170 182 194 212 229 240 249 260 294 307
 344 354 369]


In [11]:
# Using SelectKBest to get the 25 best features
selector1 = SelectKBest(k=25)

selector1.fit_transform(X_train_1, y_train_1)

feature1_25 = selector1.get_support(indices=True).tolist()
print(feature1_25)

selector2 = SelectKBest(k=25)

selector2.fit_transform(X_train_2, y_train_2)

feature2_25 = selector2.get_support(indices=True).tolist()
print(feature2_25)

selector3 = SelectKBest(k=25)

selector3.fit_transform(X_train_3, y_train_3)

feature3_25 = selector3.get_support(indices=True).tolist()
print(feature3_25)

[2, 25, 74, 75, 76, 91, 126, 127, 130, 131, 255, 256, 257, 258, 260, 275, 278, 279, 280, 281, 293, 299, 324, 326, 327]
[2, 25, 64, 91, 126, 127, 130, 131, 159, 255, 256, 257, 258, 260, 278, 279, 280, 281, 293, 299, 301, 303, 324, 326, 327]
[2, 15, 18, 25, 64, 91, 126, 127, 130, 131, 139, 159, 256, 257, 258, 278, 280, 281, 293, 299, 300, 301, 302, 303, 327]


In [12]:
# Using SelectKBest to get the 50 best features
selector1 = SelectKBest(k=50)

selector1.fit_transform(X_train_1, y_train_1)

feature1_50 = selector1.get_support(indices=True).tolist()
print(feature1_50)

selector2 = SelectKBest(k=50)

selector2.fit_transform(X_train_2, y_train_2)

feature2_50 = selector2.get_support(indices=True).tolist()
print(feature2_50)

selector3 = SelectKBest(k=50)

selector3.fit_transform(X_train_3, y_train_3)

feature3_50 = selector3.get_support(indices=True).tolist()
print(feature3_50)

[2, 5, 12, 14, 15, 17, 18, 25, 28, 29, 53, 64, 74, 75, 76, 91, 94, 95, 124, 126, 127, 128, 130, 131, 159, 255, 256, 257, 258, 259, 260, 275, 276, 278, 279, 280, 281, 292, 293, 298, 299, 300, 301, 302, 303, 312, 324, 325, 326, 327]
[2, 5, 12, 14, 15, 17, 18, 25, 28, 29, 53, 64, 74, 75, 76, 91, 94, 95, 124, 126, 127, 128, 130, 131, 139, 159, 255, 256, 257, 258, 259, 260, 275, 276, 278, 279, 280, 281, 292, 293, 298, 299, 300, 301, 302, 303, 312, 324, 326, 327]
[2, 5, 12, 13, 14, 15, 16, 17, 18, 24, 25, 28, 29, 51, 53, 54, 64, 91, 94, 95, 124, 126, 127, 128, 130, 131, 139, 159, 255, 256, 257, 258, 260, 278, 279, 280, 281, 282, 292, 293, 298, 299, 300, 301, 302, 303, 312, 324, 326, 327]


In [13]:
# Function to compare selected features
def common_features(list1, list2):
    return [element for element in list1 if element in list2]

In [14]:
common_features(col_2,feature1_50)

[128, 256, 275, 280, 302, 303, 312]

In [15]:
common_features(col_2,feature2_50)

[128, 139, 256, 275, 280, 302, 303, 312]

In [16]:
common_features(col_3,feature3_50)

[16, 130, 139, 301]

In [17]:
X_feat_25 = X_train_1.iloc[:,feature1_25]
X_feat_50 = X_train_1.iloc[:,feature1_50]

In [18]:
# Using SMOTE to balance the original dataset

smote = SMOTE(random_state = 42)

X_res, y_res = smote.fit_resample(X, y)
print(y.value_counts())
print(y_res.value_counts())

0    73012
1     3008
Name: TARGET, dtype: int64
0    73012
1    73012
Name: TARGET, dtype: int64


In [19]:
X_train_1_25 = X_res.iloc[:,feature1_25]
X_train_1_50 = X_res.iloc[:,feature1_50]

In [20]:
test_25 = test.iloc[:,feature1_25]
test_50 = test.iloc[:,feature1_50]

In [21]:
# Tuning the hyperparameters using Genetic Search for Random Forest Classifer 
clf = RandomForestClassifier()
param_grid = {'max_leaf_nodes': Integer(2, 35), 
              'n_estimators': Integer(100, 300)}

In [22]:
cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator = GASearchCV(estimator=clf,
                               cv=cv,
                               scoring='accuracy',
                               population_size=10,
                               generations=35,
                               tournament_size=3,
                               elitism=True,
                               crossover_probability=0.8,
                               mutation_probability=0.1,
                               param_grid=param_grid,
                               criteria='max',
                               algorithm='eaMuPlusLambda',
                               n_jobs=-1,
                               verbose=True,
                               keep_top_k=4)

In [23]:
# Fitting the model on the dataset with feature selection
evolved_estimator.fit(X_feat_25,y_train_1)

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.765658	0.00281731 	0.769779   	0.760306   
1  	19    	0.767287	0.00160572 	0.769779   	0.763798   
2  	20    	0.768151	0.00046303 	0.768949   	0.767453   
3  	16    	0.768217	0.000677969	0.768949   	0.766457   
4  	19    	0.768616	0.000148839	0.768949   	0.76845    
5  	18    	0.768334	0.000931669	0.770114   	0.766955   
6  	17    	0.768069	0.000724461	0.768948   	0.766956   
7  	17    	0.768118	0.000594287	0.76895    	0.767287   
8  	17    	0.768651	0.000370767	0.76895    	0.767619   
9  	16    	0.768667	0.000433651	0.769446   	0.767786   
10 	17    	0.768601	0.000582421	0.769446   	0.767123   
11 	19    	0.768617	0.000643171	0.769446   	0.767123   
12 	18    	0.769099	0.000365753	0.769446   	0.768452   
13 	19    	0.768699	0.00109037 	0.769615   	0.766621   
14 	17    	0.768949	0.000913227	0.769615   	0.767122   
15 	18    	0.768616	0.0010085  	0.769615   	0.766787   
16 	19    	0.768617	0.000697633	0.769615   	0.76

GASearchCV(crossover_probability=0.8,
           cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
           estimator=RandomForestClassifier(max_leaf_nodes=18,
                                            n_estimators=264),
           generations=35, keep_top_k=4, mutation_probability=0.1, n_jobs=-1,
           param_grid={'max_leaf_nodes': <sklearn_genetic.space.space.Integer object at 0x7fee307e4040>,
                       'n_estimators': <sklearn_genetic.space.space.Integer object at 0x7fee307e41f0>},
           population_size=10, return_train_score=True, scoring='accuracy')

In [24]:
# Hyperparameters selected using Genetic Search for Random Forest Classifier
evolved_estimator.best_params_

{'max_leaf_nodes': 18, 'n_estimators': 264}

In [25]:
# Tuning the hyperparameters using Genetic Search for Decision Tree Classifier 
clf2 = DecisionTreeClassifier()
param_grid = {'min_weight_fraction_leaf': Continuous(0.01, 0.5, distribution='log-uniform'),
              'max_leaf_nodes': Integer(2, 35), 
              'min_samples_split': Integer(2,10),
              'criterion': Categorical(['entropy', 'gini'])}

In [26]:
cv = StratifiedKFold(n_splits=3, shuffle=True)

evolved_estimator2 = GASearchCV(estimator=clf2,
                               cv=cv,
                               scoring='accuracy',
                               population_size=10,
                               generations=35,
                               tournament_size=3,
                               elitism=True,
                               crossover_probability=0.8,
                               mutation_probability=0.1,
                               param_grid=param_grid,
                               criteria='max',
                               algorithm='eaMuPlusLambda',
                               n_jobs=-1,
                               verbose=True,
                               keep_top_k=4)

In [27]:
# Fitting the model on the dataset with feature selection
evolved_estimator2.fit(X_feat_25,y_train_1)

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.748321	0.00413971 	0.756815   	0.745677   
1  	18    	0.755253	0.00339527 	0.75698    	0.745679   
2  	18    	0.756699	0.000340453	0.75698    	0.755819   
3  	19    	0.75728 	0.00065625 	0.759144   	0.756815   
4  	20    	0.757497	0.00128075 	0.759144   	0.75482    
5  	18    	0.757863	0.00110378 	0.759144   	0.756648   
6  	18    	0.757048	0.00149156 	0.759144   	0.754987   
7  	19    	0.757165	0.00135811 	0.759144   	0.75532    
8  	19    	0.757231	0.00141184 	0.759144   	0.755484   
9  	17    	0.757746	0.00116958 	0.759144   	0.75615    
10 	17    	0.757763	0.00113815 	0.759144   	0.756648   
11 	18    	0.757879	0.00107649 	0.759144   	0.756649   
12 	16    	0.757431	0.000992374	0.759144   	0.756152   
13 	18    	0.758029	0.00076605 	0.759144   	0.756981   
14 	19    	0.757962	0.00096533 	0.759144   	0.756317   
15 	19    	0.75763 	0.00148254 	0.759144   	0.755319   
16 	18    	0.758196	0.00129745 	0.759144   	0.75

GASearchCV(crossover_probability=0.8,
           cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=True),
           estimator=DecisionTreeClassifier(criterion='entropy',
                                            max_leaf_nodes=33,
                                            min_samples_split=10,
                                            min_weight_fraction_leaf=0.03660952020509553),
           generations=35, keep_top_k=4, mutation_probability=0.1, n_jobs=-1,
           param_grid={'criterion': <sklearn_genetic.space.space.Categorical object at 0x7fee0182eeb0>,
                       'max_leaf_nodes': <sklearn_genetic.space.space.Integer object at 0x7fee0182e8e0>,
                       'min_samples_split': <sklearn_genetic.space.space.Integer object at 0x7fee0182e9a0>,
                       'min_weight_fraction_leaf': <sklearn_genetic.space.space.Continuous object at 0x7fee25cd7400>},
           population_size=10, return_train_score=True, scoring='accuracy')

In [28]:
# Hyperparameter selected using Genetic Search for Decision Tree Classifier
evolved_estimator2.best_params_

{'min_weight_fraction_leaf': 0.03660952020509553,
 'max_leaf_nodes': 33,
 'min_samples_split': 10,
 'criterion': 'entropy'}

In [29]:
# Initializing models using the hyperparameters obtained
model1 = RandomForestClassifier(max_leaf_nodes=18, n_estimators=264)

model2 = DecisionTreeClassifier(min_weight_fraction_leaf=0.03660952020509553,
 max_leaf_nodes=33,
 min_samples_split=10,
 criterion="entropy")

In [30]:
# Predicting the models on the different test datasets
model1.fit(X_train_1_25,y_res)
pred_rf_25 = model1.predict_proba(test_25)

model1.fit(X_train_1_50,y_res)
pred_rf_50 = model1.predict_proba(test_50)

In [31]:
# Predicting the models on the different test datasets
model2.fit(X_train_1_25,y_res)
pred_dt_25 = model2.predict_proba(test_25)

model2.fit(X_train_1_50,y_res)
pred_dt_50 = model2.predict_proba(test_50)

In [32]:
# Preparing the submission files for Kaggle
train_df = train.copy()
test_df = test.copy()
train_id = train_df['ID']
test_id = test['ID']
train_df.drop('ID', axis=1, inplace=True)
test_df.drop('ID', axis=1, inplace=True)
test_df = test_df[X_train_1_25.columns]

In [33]:
test_df_1 = test_df[X_train_1_25.columns]

submission = pd.DataFrame({"ID":test_id, "TARGET": pred_dt_25[:,1]})
submission.to_csv("submission.csv", index=False)

In [34]:
submission2 = pd.DataFrame({"ID":test_id, "TARGET": pred_rf_25[:,1]})
submission2.to_csv("submission2.csv", index=False)

In [35]:
submission3 = pd.DataFrame({"ID":test_id, "TARGET": pred_dt_50[:,1]})
submission3.to_csv("submission3.csv", index=False)

In [36]:
submission4 = pd.DataFrame({"ID":test_id, "TARGET": pred_rf_50[:,1]})
submission4.to_csv("submission4.csv", index=False)